In [1]:
from IPython.display import HTML
HTML(open('../style.css').read())

# The Bridge and the Torch

In the darkness of the night, a group of four individuals encounters a river. A slender bridge stretches before them, capable of accommodating just two people simultaneously. Equipped with a single torch, they must rely on its flickering light to navigate the bridge. Each person possesses a distinct crossing time: Ariela takes 1 minute, Brian takes 2 minutes, Charly takes 5 minutes, and Dumpy takes 8 minutes. It is crucial to note that when two people cross together, they must synchronize their steps with the slower individual's pace. Given the torch's limited lifespan of 15 minutes, the pressing question arises: can all four individuals successfully traverse the bridge?

You can find an animation here:
http://www.youtube.com/watch?v=7yDmGnA8Hw0



In [2]:
import z3

We need the following variables to encode the problem:
* `A` equals `1` if Ariela is on the left shore,
* `B` equals `1` if Brian  is on the left shore,
* `C` equals `1` if Charly is on the left shore,
* `D` equals `1` if Dumpy  is on the left shore,
* `T` equals the time that has passed.

`start` returns a set of constraints that specify that everybody is on the left side of the river.

In [3]:
def start(A, B, C, D, T):
    return z3.And(A == 1, B == 1, C == 1, D == 1, T == 0)

`goal` returns a set of constraints that specify that everybody is on the right side of the river.

In [4]:
def goal(A, B, C, D, T):
    return z3.And(A == 0, B == 0, C == 0, D == 0, T == 15)

`transition` returns a set of constraints that describe the crossings of the bridge.
The variable `i` specifies the number of the crossing.  The first crossing has `i == 0`.

In [6]:


def transition(A, B, C, D, T, Ax, Bx, Cx, Dx, Tx, i):
    
    # helper: did X move this step?
    movedA = z3.If(A != Ax, 1, 0)
    movedB = z3.If(B != Bx, 1, 0)
    movedC = z3.If(C != Cx, 1, 0)
    movedD = z3.If(D != Dx, 1, 0)

    # how many actually moved?
    movers = movedA + movedB + movedC + movedD

    # direction = +1 for L→R, -1 for R→L
    left_to_right = (i % 2 == 0)

    # legal-move skeleton shared by all cases
    base = lambda: z3.And(
        # variables stay Boolean
        *(v >= 0 for v in (A,B,C,D,Ax,Bx,Cx,Dx)),
        *(v <= 1 for v in (A,B,C,D,Ax,Bx,Cx,Dx)),
        # correct number of travellers
        movers == (2 if left_to_right else 1)
    )

    # *** build concrete cases ************************************************
    cases = []

    # list of people present on the *correct* starting shore
    present_left  = [(A,Ax,'A'), (B,Bx,'B'), (C,Cx,'C'), (D,Dx,'D')]
    present_right = [(1-a,1-ax,n) for (a,ax,n) in present_left]  # mirror

    start_side = present_left  if left_to_right else present_right

    for idx1 in range(4):
        p1, p1x, n1 = start_side[idx1]
        # single-traveller cases (allowed only when torch is on the right)
        if not left_to_right:
            time = times[n1]
            cases.append(z3.And(
                base(),
                # he really moved
                p1 != p1x,
                # others stayed
                *(q == qx for j,(q,qx,_) in enumerate(start_side) if j!=idx1),
                Tx == T + time
            ))

        # two-traveller cases (allowed only when torch is on the left)
        if left_to_right:
            for idx2 in range(idx1+1, 4):
                p2, p2x, n2 = start_side[idx2]
                time = max(times[n1], times[n2])
                cases.append(z3.And(
                    base(),
                    p1 != p1x,  p2 != p2x,                   # these two move
                    *(q == qx for j,(q,qx,_) in enumerate(start_side)
                       if j not in (idx1, idx2)),           # rest stay
                    Tx == T + time
                ))

    return z3.Or(*cases) 
        

In [7]:
def bridge_CSP(n):
    S = z3.Solver()
    A = [z3.Int(f'A{i}') for i in range(n+1)]
    B = [z3.Int(f'B{i}') for i in range(n+1)]
    C = [z3.Int(f'C{i}') for i in range(n+1)]
    D = [z3.Int(f'D{i}') for i in range(n+1)]
    T = [z3.Int(f'T{i}') for i in range(n+1)]
    Cts  = start(A[0], B[0], C[0], D[0], T[0])
    Cts |= goal( A[n], B[n], C[n], D[n], T[n])
    for i in range(n):
        j = i+1
        Cts |= transition(A[i], B[i], C[i], D[i], T[i],
                          A[j], B[j], C[j], D[j], T[j],
                          i)
        Cts.add(0 <= A[i])
        Cts.add(0 <= B[i])
        Cts.add(0 <= C[i])
        Cts.add(0 <= D[i])
        Cts.add(0 <= T[i])
        Cts.add(A[i] <= 1) 
        Cts.add(B[i] <= 1)
        Cts.add(C[i] <= 1)
        Cts.add(D[i] <= 1)
    S.add(Cts)
    result = str(S.check())
    if result == 'sat':
        Model = S.model()
        Solution = (   { f'A{i}': Model[A[i]] for i in range(n+1) }
                     | { f'B{i}': Model[B[i]] for i in range(n+1) }
                     | { f'C{i}': Model[D[i]] for i in range(n+1) }
                     | { f'D{i}': Model[C[i]] for i in range(n+1) }
                     | { f'T{i}': Model[T[i]] for i in range(n+1) }
                   )
        return { key: Solution[key].as_long() for key in Solution }
    else:
        return None

In [8]:
def find_solution():
    n = 1
    while True:
        print(n)
        Solution = bridge_CSP(n)
        if Solution is not None:
            return n, Solution
        n += 2

In [9]:
%%time
n, Solution = find_solution()
n, Solution

1


NameError: name 'times' is not defined

In [ ]:
def show_solution(Solution, n):
    for i in range(n+1):
        A = Solution[f'A{i}']
        B = Solution[f'B{i}']
        C = Solution[f'C{i}']
        D = Solution[f'D{i}']
        T = Solution[f'T{i}']
        print('🏃‍♀️'*A+'🏃🏽‍♂️'*B+'🚶🏽‍♂️'*C+'👨‍🦽'*D + ' '*42 + \
              '🏃‍♀️'*(1-A)+'🏃🏽‍♂️'*(1-B)+'🚶🏽‍♂️'*(1-C)+'👨‍🦽'*(1-D))
        print(f'🕰️{T}')
        if i % 2 == 0:
            PS = Solution[f'A{i}'] - Solution[f'A{i+1}']
            BS = Solution[f'B{i}'] - Solution[f'B{i+1}']
            CS = Solution[f'C{i}'] - Solution[f'C{i+1}']
            FS = Solution[f'D{i}'] - Solution[f'D{i+1}']
            TS = Solution[f'T{i+1}']
            print(' '*24+'>> '+'🏃‍♀️'*PS+'🏃🏽‍♂️'*BS+'🚶🏽‍♂️'*CS+'👨‍🦽'*FS+' >>')
        elif i + 1 < n:
            PS = Solution[f'A{i+1}'] - Solution[f'A{i}']
            BS = Solution[f'B{i+1}'] - Solution[f'B{i}']
            CS = Solution[f'C{i+1}'] - Solution[f'C{i}']
            FS = Solution[f'D{i+1}'] - Solution[f'D{i}']
            TS = Solution[f'T{i}']
            print(' '*24+'<< '+'🏃‍♀️'*PS+'🏃🏽‍♂️'*BS+'🚶🏽‍♂️'*CS+'👨‍🦽'*FS+' <<')

In [ ]:
show_solution(Solution, n)